# 🧠 V4 — Hybrid Residual Learning (XGBoost + DL)
### Strategy: DL learns what XGBoost gets wrong
1. Generate XGBoost predictions for all windows
2. Compute residuals: `residual = actual - XGBoost_pred`
3. Train DL model to predict these residuals from input sequences
4. Final prediction = `XGBoost_pred + DL_residual_pred`

**Why this works:**
- XGBoost already captures 85%+ of the signal (MAE=2120)
- DL only needs to learn the remaining ~15% (temporal patterns XGBoost misses)
- Residuals are small & centered around 0 → much easier to learn
- No under-prediction problem (residuals are symmetric)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, time, math, copy, numpy as np, joblib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings; warnings.filterwarnings('ignore')

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)): return int(obj)
        if isinstance(obj, (np.floating,)): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super().default(obj)

BASE_DIR  = '/content/drive/MyDrive/Electricity_Load_Forecast'
MODEL_DIR = os.path.join(BASE_DIR, 'models')
V4_DIR    = os.path.join(MODEL_DIR, 'v4')
os.makedirs(V4_DIR, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
config        = joblib.load(os.path.join(MODEL_DIR, 'config.pkl'))
target_scaler = joblib.load(os.path.join(MODEL_DIR, 'target_scaler.pkl'))
X_train_w = np.load(os.path.join(MODEL_DIR, 'X_train_w.npy'))
y_train_w = np.load(os.path.join(MODEL_DIR, 'y_train_w.npy'))
X_val_w   = np.load(os.path.join(MODEL_DIR, 'X_val_w.npy'))
y_val_w   = np.load(os.path.join(MODEL_DIR, 'y_val_w.npy'))
X_test_w  = np.load(os.path.join(MODEL_DIR, 'X_test_w.npy'))
y_test_w  = np.load(os.path.join(MODEL_DIR, 'y_test_w.npy'))
LOAD_IDX   = config['load_col_idx']
N_FEATURES = config['N_FEATURES']
INPUT_LEN  = config['INPUT_LEN']
OUTPUT_LEN = config['OUTPUT_LEN']
print(f'Features={N_FEATURES}, In={INPUT_LEN}h, Out={OUTPUT_LEN}h, LoadIdx={LOAD_IDX}')
print(f'Train={X_train_w.shape}, Val={X_val_w.shape}, Test={X_test_w.shape}')

In [ ]:
# ═══════════════════════════════════════
#  STEP 1: GET XGBOOST PREDICTIONS
# ═══════════════════════════════════════
def engineer_features(X_windows, load_idx):
    """Same feature extraction as baseline_model notebook."""
    feat_list = []
    for w in X_windows:
        f = []
        load = w[:, load_idx]
        f.extend([load.mean(), load.std(), load.min(), load.max(),
                  load[-1], load[0], load[-1] - load[0],
                  load[-24:].mean(), load[-48:].mean()])
        for j in range(w.shape[1]):
            if j != load_idx:
                f.extend([w[:, j].mean(), w[-1, j]])
        feat_list.append(f)
    return np.array(feat_list, dtype=np.float32)

# Load saved XGBoost model
print('Loading XGBoost model...')
xgb_model = joblib.load(os.path.join(MODEL_DIR, 'xgb_model.pkl'))

# Extract features & predict
print('Generating XGBoost predictions for all windows...')
t0 = time.time()
Xf_train = engineer_features(X_train_w, LOAD_IDX)
Xf_val   = engineer_features(X_val_w,   LOAD_IDX)
Xf_test  = engineer_features(X_test_w,  LOAD_IDX)

xgb_pred_train = xgb_model.predict(Xf_train)  # (N_train, 168)
xgb_pred_val   = xgb_model.predict(Xf_val)
xgb_pred_test  = xgb_model.predict(Xf_test)
print(f'Done in {time.time()-t0:.1f}s')
print(f'XGBoost pred shapes: train={xgb_pred_train.shape}, val={xgb_pred_val.shape}, test={xgb_pred_test.shape}')

In [ ]:
# ═══════════════════════════════════════
#  STEP 2: COMPUTE RESIDUALS
# ═══════════════════════════════════════
# residual = actual - xgboost_prediction (in scaled space)
res_train = y_train_w - xgb_pred_train
res_val   = y_val_w   - xgb_pred_val
res_test  = y_test_w  - xgb_pred_test

print(f'Residual stats (scaled):')
print(f'  Train — mean: {res_train.mean():.4f}, std: {res_train.std():.4f}')
print(f'  Val   — mean: {res_val.mean():.4f}, std: {res_val.std():.4f}')
print(f'  Test  — mean: {res_test.mean():.4f}, std: {res_test.std():.4f}')

# Visualize residual distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, data, title in zip(axes, [res_train, res_val, res_test], ['Train','Val','Test']):
    ax.hist(data.flatten(), bins=100, alpha=0.7, color='steelblue')
    ax.axvline(0, color='red', ls='--')
    ax.set_title(f'{title} Residuals'); ax.set_xlabel('Residual (scaled)')
plt.suptitle('XGBoost Residual Distribution (what DL needs to learn)', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V4_DIR, 'residual_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  STEP 3: PREPARE DATA FOR DL
# ═══════════════════════════════════════
HP = {
    'batch_size': 64, 'lr': 3e-4, 'weight_decay': 1e-3,
    'max_epochs': 80, 'patience': 20, 'grad_clip': 0.5,
    'dropout': 0.25, 'noise_std': 0.015,
    'mixup_alpha': 0.2,
    'conv_filters': 48, 'lstm_hidden': 128, 'n_heads': 4,
    'n_ensemble': 3, 'seeds': [42, 123, 456],
}
print('HP:', json.dumps(HP, indent=2))

# Include XGBoost prediction as extra input feature
# Shape: (N, 168, N_FEATURES + 1) — original features + XGBoost pred
def add_xgb_feature(X, xgb_pred):
    return np.concatenate([X, xgb_pred[:, :, None]], axis=-1).astype(np.float32)

X_train_aug = add_xgb_feature(X_train_w, xgb_pred_train)
X_val_aug   = add_xgb_feature(X_val_w,   xgb_pred_val)
X_test_aug  = add_xgb_feature(X_test_w,  xgb_pred_test)
N_FEAT_AUG  = X_train_aug.shape[-1]
print(f'Augmented input shape: {X_train_aug.shape} (added XGBoost pred as feature #{N_FEAT_AUG})')

def make_loader(X, y, bs, shuffle=True):
    ds = TensorDataset(torch.tensor(X, dtype=torch.float32),
                       torch.tensor(y, dtype=torch.float32))
    return DataLoader(ds, batch_size=bs, shuffle=shuffle,
                      num_workers=2, pin_memory=True, drop_last=shuffle)

# DL targets are RESIDUALS, not raw values
train_loader = make_loader(X_train_aug, res_train, HP['batch_size'])
val_loader   = make_loader(X_val_aug,   res_val,   HP['batch_size'], shuffle=False)
test_loader  = make_loader(X_test_aug,  res_test,  HP['batch_size'], shuffle=False)
print(f'Batches — Train:{len(train_loader)}, Val:{len(val_loader)}, Test:{len(test_loader)}')

In [ ]:
# ═══════════════════════════════════════
#  STEP 4: RESIDUAL DL MODEL
# ═══════════════════════════════════════
class LightCNN(nn.Module):
    def __init__(self, ch, dropout=0.15):
        super().__init__()
        self.conv3 = nn.Conv1d(ch, ch, 3, padding=1)
        self.conv7 = nn.Conv1d(ch, ch, 7, padding=3)
        self.bn = nn.BatchNorm1d(ch * 2)
        self.proj = nn.Conv1d(ch * 2, ch, 1)
        self.bn2 = nn.BatchNorm1d(ch)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        c3 = F.gelu(self.conv3(x))
        c7 = F.gelu(self.conv7(x))
        out = F.gelu(self.bn(torch.cat([c3, c7], dim=1)))
        return self.drop(F.gelu(self.bn2(self.proj(out))))


class ResidualPredictor(nn.Module):
    """Predicts XGBoost residuals from enriched input sequences.
    Input: (B, 168, N_FEATURES+1) where last feature = XGBoost prediction.
    Output: (B, 168) predicted residual."""
    def __init__(self, n_features, pred_len,
                 conv_filters=48, lstm_hidden=128, n_heads=4,
                 dropout=0.25, noise_std=0.015):
        super().__init__()
        self.noise_std = noise_std
        d = lstm_hidden * 2

        self.input_proj = nn.Sequential(
            nn.Linear(n_features, conv_filters),
            nn.LayerNorm(conv_filters), nn.GELU(),
            nn.Dropout(dropout * 0.5))

        self.cnn = LightCNN(conv_filters, dropout * 0.5)

        self.lstm = nn.LSTM(conv_filters, lstm_hidden,
                            batch_first=True, bidirectional=True)
        self.ln1 = nn.LayerNorm(d)
        self.drop = nn.Dropout(dropout)

        self.attn = nn.MultiheadAttention(d, n_heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(d)
        self.ffn = nn.Sequential(nn.Linear(d, d*2), nn.GELU(),
                                 nn.Dropout(dropout), nn.Linear(d*2, d))
        self.ln3 = nn.LayerNorm(d)

        # Per-timestep output → predicted residual
        self.head = nn.Sequential(
            nn.Linear(d, d // 2), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(d // 2, 1))

    def forward(self, x):
        if self.training and self.noise_std > 0:
            x = x + torch.randn_like(x) * self.noise_std
        x = self.input_proj(x)
        x = self.cnn(x.permute(0,2,1)).permute(0,2,1)
        h, _ = self.lstm(x)
        h = self.drop(self.ln1(h))
        a, _ = self.attn(h, h, h)
        h = self.ln2(h + a)
        h = self.ln3(h + self.ffn(h))
        return self.head(h).squeeze(-1)  # (B, 168)

_m = ResidualPredictor(N_FEAT_AUG, OUTPUT_LEN, HP['conv_filters'],
                       HP['lstm_hidden'], HP['n_heads'], HP['dropout'])
print(f'ResidualPredictor params: {sum(p.numel() for p in _m.parameters()):,}')
del _m

In [ ]:
# ═══════════════════════════════════════
#  STEP 5: TRAINING FUNCTION
# ═══════════════════════════════════════
def mixup(X, y, alpha=0.2):
    if alpha <= 0: return X, y
    lam = max(np.random.beta(alpha, alpha), 0.5)
    idx = torch.randperm(X.size(0), device=X.device)
    return lam*X + (1-lam)*X[idx], lam*y + (1-lam)*y[idx]

def train_residual_model(seed, HP, train_loader, val_loader, device, save_dir, idx):
    torch.manual_seed(seed); np.random.seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed(seed)

    model = ResidualPredictor(
        N_FEAT_AUG, OUTPUT_LEN, HP['conv_filters'],
        HP['lstm_hidden'], HP['n_heads'], HP['dropout'], HP['noise_std']
    ).to(device)

    # MSE loss for residuals (they're centered around 0, symmetric)
    criterion = nn.HuberLoss(delta=0.5)
    opt  = torch.optim.AdamW(model.parameters(), lr=HP['lr'],
                             weight_decay=HP['weight_decay'])
    sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        opt, T_0=20, T_mult=2, eta_min=1e-6)

    hist = {'train': [], 'val': [], 'lr': []}
    best_vl, patience_ctr, best_state = float('inf'), 0, None

    print(f"\n{'─'*45}")
    print(f"  Model {idx+1}/{HP['n_ensemble']}  seed={seed}")
    print(f"{'─'*45}")

    for ep in range(HP['max_epochs']):
        t0 = time.time()
        model.train(); losses = []
        for X_b, y_b in train_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            X_b, y_b = mixup(X_b, y_b, HP['mixup_alpha'])
            opt.zero_grad()
            loss = criterion(model(X_b), y_b)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), HP['grad_clip'])
            opt.step()
            losses.append(loss.item())
        sched.step()

        model.eval(); vl = []
        with torch.no_grad():
            for X_b, y_b in val_loader:
                vl.append(criterion(model(X_b.to(device)), y_b.to(device)).item())

        tl = float(np.mean(losses)); vl = float(np.mean(vl))
        lr = float(opt.param_groups[0]['lr'])
        hist['train'].append(tl); hist['val'].append(vl); hist['lr'].append(lr)

        tag = ''
        if vl < best_vl:
            best_vl, patience_ctr = vl, 0
            best_state = copy.deepcopy(model.state_dict())
            tag = ' ✅'
        else:
            patience_ctr += 1

        if (ep+1) % 5 == 0 or tag:
            print(f'  Ep {ep+1:3d}/{HP["max_epochs"]} | '
                  f'T:{tl:.5f} V:{vl:.5f} LR:{lr:.1e} {time.time()-t0:.1f}s{tag}')

        if patience_ctr >= HP['patience']:
            print(f'  ⛔ Early stop ep {ep+1}'); break

    model.load_state_dict(best_state)
    torch.save(model.state_dict(), os.path.join(save_dir, f'res_model_{idx}_s{seed}.pt'))
    print(f'  ✅ Done (best val={best_vl:.5f})')
    return model, hist, best_vl

In [ ]:
# ═══════════════════════════════════════
#  STEP 6: TRAIN ENSEMBLE
# ═══════════════════════════════════════
models, histories, val_losses = [], [], []
for i, seed in enumerate(HP['seeds'][:HP['n_ensemble']]):
    m, h, vl = train_residual_model(seed, HP, train_loader, val_loader, device, V4_DIR, i)
    models.append(m); histories.append(h); val_losses.append(vl)

print(f"\n{'='*50}")
print(f'  ENSEMBLE DONE — val losses: {[round(v,5) for v in val_losses]}')
print(f"{'='*50}")

In [ ]:
# Training history
fig, axes = plt.subplots(1, HP['n_ensemble'], figsize=(6*HP['n_ensemble'], 5))
if HP['n_ensemble'] == 1: axes = [axes]
for i, (h, ax) in enumerate(zip(histories, axes)):
    ax.plot(h['train'], label='Train', color='#2196F3')
    ax.plot(h['val'],   label='Val',   color='#FF5722')
    ax.set_title(f'Residual Model {i+1} (seed={HP["seeds"][i]})')
    ax.set_xlabel('Epoch'); ax.set_ylabel('Huber Loss')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('V4 Residual Models — Training History', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V4_DIR, 'training_history_v4.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  STEP 7: COMBINE XGBoost + DL
# ═══════════════════════════════════════
def predict_residuals(model, loader):
    model.eval(); ps = []
    with torch.no_grad():
        for X_b, _ in loader:
            ps.append(model(X_b.to(device)).cpu().numpy())
    return np.concatenate(ps)

def ensemble_residuals(models, loader):
    return np.mean([predict_residuals(m, loader) for m in models], axis=0)

def inv(y):
    return target_scaler.inverse_transform(y.reshape(-1,1)).reshape(y.shape)

def compute_metrics(y_true_scaled, y_pred_scaled):
    yt, yp = inv(y_true_scaled), inv(y_pred_scaled)
    mae  = float(mean_absolute_error(yt.flat, yp.flat))
    rmse = float(np.sqrt(mean_squared_error(yt.flat, yp.flat)))
    mask = yt.flat[:] != 0
    mape = float(np.mean(np.abs((yt.flat[mask]-yp.flat[mask])/yt.flat[mask]))*100)
    pk = [float(np.abs(yt[i,np.argmax(yt[i])]-yp[i,np.argmax(yt[i])])) for i in range(len(yt))]
    return {'MAE':round(mae,2),'RMSE':round(rmse,2),'MAPE':round(mape,2),'Peak_MAE':round(float(np.mean(pk)),2)}

# Get predicted residuals
res_pred_val  = ensemble_residuals(models, val_loader)
res_pred_test = ensemble_residuals(models, test_loader)

# HYBRID prediction = XGBoost + DL residual
hybrid_pred_val  = xgb_pred_val[:len(res_pred_val)]   + res_pred_val
hybrid_pred_test = xgb_pred_test[:len(res_pred_test)] + res_pred_test

# Trim targets to match
y_val_trim  = y_val_w[:len(res_pred_val)]
y_test_trim = y_test_w[:len(res_pred_test)]

# XGBoost-only metrics
xgb_val_m  = compute_metrics(y_val_trim,  xgb_pred_val[:len(res_pred_val)])
xgb_test_m = compute_metrics(y_test_trim, xgb_pred_test[:len(res_pred_test)])

# Hybrid metrics
hyb_val_m  = compute_metrics(y_val_trim,  hybrid_pred_val)
hyb_test_m = compute_metrics(y_test_trim, hybrid_pred_test)

print(f"\n{'='*50}\nXGBoost Only — Val:  {xgb_val_m}")
print(f"XGBoost Only — Test: {xgb_test_m}")
print(f"\nHybrid (XGB+DL) — Val:  {hyb_val_m}")
print(f"Hybrid (XGB+DL) — Test: {hyb_test_m}\n{'='*50}")

# Improvement
print(f"\n{'='*50}")
print(f'   HYBRID IMPROVEMENT vs XGBoost')
print(f"{'='*50}")
for k in ['MAE','RMSE','MAPE','Peak_MAE']:
    xv, hv = xgb_test_m[k], hyb_test_m[k]
    pct = (xv - hv) / xv * 100
    arr = '✅↓' if pct > 0 else '❌↑'
    print(f'  {k:10s}: XGB={xv:>8} → Hybrid={hv:>8}  ({pct:+.1f}%) {arr}')

In [ ]:
# ═══════════════════════════════════════
#  STEP 8: WEIGHTED BLENDING (optimize alpha)
# ═══════════════════════════════════════
# Try different blend weights: final = alpha * hybrid + (1-alpha) * xgboost
# to find the optimal blend on validation set
best_alpha, best_mae = 1.0, float('inf')
alphas_tried = np.arange(0.0, 1.05, 0.05)
alpha_results = []

for alpha in alphas_tried:
    blended = alpha * hybrid_pred_val + (1 - alpha) * xgb_pred_val[:len(res_pred_val)]
    m = compute_metrics(y_val_trim, blended)
    alpha_results.append((float(alpha), m['MAE']))
    if m['MAE'] < best_mae:
        best_mae = m['MAE']
        best_alpha = float(alpha)

print(f'Best blend alpha: {best_alpha:.2f} (val MAE: {best_mae:.2f})')
print(f'  alpha=1.0 means full hybrid, alpha=0.0 means pure XGBoost')

plt.figure(figsize=(10, 4))
plt.plot([a for a,_ in alpha_results], [m for _,m in alpha_results], 'bo-')
plt.axvline(best_alpha, color='red', ls='--', label=f'Best α={best_alpha:.2f}')
plt.xlabel('Alpha (blend weight)'); plt.ylabel('Val MAE')
plt.title('Blend Weight Optimization'); plt.legend(); plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(V4_DIR, 'blend_optimization.png'), dpi=150)
plt.show()

# Apply best blend to test
final_pred_test = best_alpha * hybrid_pred_test + (1 - best_alpha) * xgb_pred_test[:len(res_pred_test)]
final_test_m = compute_metrics(y_test_trim, final_pred_test)

final_pred_val = best_alpha * hybrid_pred_val + (1 - best_alpha) * xgb_pred_val[:len(res_pred_val)]
final_val_m = compute_metrics(y_val_trim, final_pred_val)

print(f"\n{'='*50}")
print(f'   FINAL (Optimally Blended) — Test: {final_test_m}')
print(f"{'='*50}")

print(f"\n  FINAL IMPROVEMENT vs XGBoost:")
for k in ['MAE','RMSE','MAPE','Peak_MAE']:
    xv, fv = xgb_test_m[k], final_test_m[k]
    pct = (xv - fv) / xv * 100
    arr = '✅↓' if pct > 0 else '❌↑'
    print(f'    {k:10s}: XGB={xv:>8} → Final={fv:>8}  ({pct:+.1f}%) {arr}')

In [ ]:
# ═══════════════════════════════════════
#  HORIZON-WISE ERROR
# ═══════════════════════════════════════
tt_inv = inv(y_test_trim)
fp_inv = inv(final_pred_test)
xp_inv = inv(xgb_pred_test[:len(res_pred_test)])

horizons = [1, 24, 72, 168]
hmets = {}
for h in horizons:
    i = h - 1
    m_h = float(mean_absolute_error(tt_inv[:,i], fp_inv[:,i]))
    r_h = float(np.sqrt(mean_squared_error(tt_inv[:,i], fp_inv[:,i])))
    mk = tt_inv[:,i] != 0
    mp_h = float(np.mean(np.abs((tt_inv[:,i][mk]-fp_inv[:,i][mk])/tt_inv[:,i][mk]))*100)
    hmets[h] = {'MAE':round(m_h,2),'RMSE':round(r_h,2),'MAPE':round(mp_h,2)}

print(f"{'Hour':>6} {'MAE':>10} {'RMSE':>10} {'MAPE%':>10}")
for h,m in hmets.items(): print(f'{h:>6} {m["MAE"]:>10} {m["RMSE"]:>10} {m["MAPE"]:>10}')

# Compare horizon-wise: XGBoost vs Hybrid
mph_hybrid = [float(mean_absolute_error(tt_inv[:,h], fp_inv[:,h])) for h in range(OUTPUT_LEN)]
mph_xgb    = [float(mean_absolute_error(tt_inv[:,h], xp_inv[:,h])) for h in range(OUTPUT_LEN)]

plt.figure(figsize=(12, 5))
plt.plot(range(1,OUTPUT_LEN+1), mph_xgb, 'r-', lw=1.5, label='XGBoost', alpha=0.7)
plt.plot(range(1,OUTPUT_LEN+1), mph_hybrid, 'g-', lw=1.5, label='V4 Hybrid')
for h in horizons:
    plt.axvline(h, color='gray', ls=':', alpha=0.4)
plt.xlabel('Horizon (h)'); plt.ylabel('MAE (MW)')
plt.title('Horizon-Wise MAE: XGBoost vs V4 Hybrid'); plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(V4_DIR, 'horizon_wise_v4.png'), dpi=150)
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  FULL MODEL COMPARISON
# ═══════════════════════════════════════
with open(os.path.join(MODEL_DIR, 'baseline_metrics.json')) as f:
    bl = json.load(f)
v1, v2 = {}, {}
try:
    with open(os.path.join(MODEL_DIR, 'dl_metrics.json')) as f:
        raw = json.load(f)
    v1 = raw.get('CNN-BiLSTM-Attn', {'val':raw.get('val',{}),'test':raw.get('test',{})})
except: pass
try:
    with open(os.path.join(MODEL_DIR,'v2','dl_metrics_v2.json')) as f:
        raw2 = json.load(f)
    v2 = {'val':raw2.get('val',{}),'test':raw2.get('test',{})}
except: pass
v3 = {}
try:
    with open(os.path.join(MODEL_DIR,'v3','dl_metrics_v3.json')) as f:
        raw3 = json.load(f)
    v3 = {'val':raw3.get('ensemble_val',{}),'test':raw3.get('corrected_test',{})}
except: pass

all_m = {'Persistence': bl['Persistence'], 'XGBoost': bl['XGBoost']}
if v1: all_m['DL V1'] = v1
if v2: all_m['DL V2'] = v2
if v3: all_m['DL V3'] = v3
all_m['V4 Hybrid'] = {'val': final_val_m, 'test': final_test_m}

print(f"\n{'='*75}")
print(f'   FULL MODEL COMPARISON (ALL VERSIONS)')
print(f"{'='*75}")
for sp in ['val','test']:
    print(f"\n--- {sp.upper()} ---")
    print(f"{'Model':<18} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
    print('-'*58)
    for n,m in all_m.items():
        if sp in m and m[sp]:
            t=m[sp]
            print(f"{n:<18} {t['MAE']:>8} {t['RMSE']:>8} {t['MAPE']:>8} {t['Peak_MAE']:>10}")

# Bar chart
fig, axes = plt.subplots(1, 4, figsize=(22, 5))
mns = [n for n in all_m if 'test' in all_m[n] and all_m[n]['test']]
colors = ['#e74c3c','#3498db','#95a5a6','#f39c12','#9b59b6','#2ecc71']
for ax, mk in zip(axes, ['MAE','RMSE','MAPE','Peak_MAE']):
    vals = [all_m[n]['test'][mk] for n in mns]
    bars = ax.bar(range(len(mns)), vals, color=colors[:len(mns)], edgecolor='white')
    ax.set_title(mk, fontweight='bold'); ax.set_ylabel(mk)
    ax.set_xticks(range(len(mns))); ax.set_xticklabels(mns, rotation=25, ha='right', fontsize=8)
    for b,v in zip(bars,vals):
        ax.text(b.get_x()+b.get_width()/2, b.get_height()+0.5, f'{v}', ha='center', fontsize=8)
    ax.grid(True, alpha=0.2, axis='y')
plt.suptitle('All Models — Test Set', fontsize=14, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(V4_DIR, 'model_comparison_v4.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  SAVE ALL ARTIFACTS
# ═══════════════════════════════════════
import pandas as pd

v4_full = {
    'xgb_only_test': xgb_test_m,
    'hybrid_test': hyb_test_m,
    'final_blended_val': final_val_m,
    'final_blended_test': final_test_m,
    'best_blend_alpha': best_alpha,
    'horizon_wise': {str(k):v for k,v in hmets.items()},
    'individual_val_losses': [round(float(v),5) for v in val_losses],
    'hyperparameters': HP,
}
with open(os.path.join(V4_DIR, 'dl_metrics_v4.json'), 'w') as f:
    json.dump(v4_full, f, indent=2, cls=NumpyEncoder)

for i,h in enumerate(histories):
    pd.DataFrame(h).to_csv(os.path.join(V4_DIR, f'history_model{i}.csv'), index=False)

comp_save = {k:v for k,v in all_m.items() if 'test' in v and v['test']}
with open(os.path.join(V4_DIR, 'all_model_comparison_v4.json'), 'w') as f:
    json.dump(comp_save, f, indent=2, cls=NumpyEncoder)

print('\n✅ All V4 artifacts saved to models/v4/')

In [ ]:
# Sample predictions: XGBoost vs Hybrid vs Actual
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
n = len(final_pred_test)
for idx, ax in enumerate(axes.flat):
    si = idx * (n // 4)
    ax.plot(tt_inv[si], 'k-', lw=2, label='Actual')
    ax.plot(xp_inv[si], 'r--', lw=1.2, alpha=0.7, label='XGBoost')
    ax.plot(fp_inv[si], 'g-', lw=1.5, label='V4 Hybrid')
    ax.set_title(f'Test #{si}'); ax.set_xlabel('Hour'); ax.set_ylabel('MW')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('V4 Hybrid Predictions: XGBoost vs Hybrid vs Actual', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V4_DIR, 'sample_predictions_v4.png'), dpi=150, bbox_inches='tight')
plt.show()
print('🎉 V4 complete!')